# web scraping from wikipedia using beautifulsoup

# Introduction

Showing the investor what neighbourhood is famous for what and helping him make a riskfree decision of investing. we are only looking at the neighbouhoods of Toronto in this notebook. we will use our data science techniques to clean data and get something useful out of it.

## Data

* using webscraping to scrap data from wikipedia about toronto neighbourhood
* using FORESQUARE API to get the 10 most visited places, their categories and other information


In [1]:
from bs4 import BeautifulSoup


In [2]:
import requests
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


In [3]:
source= requests.get(url).text


In [4]:
soup= BeautifulSoup(source,'lxml')


In [5]:
table=soup.find('table')


In [6]:
import pandas as pd


In [7]:
tables = pd.read_html(source)


In [8]:
df = pd.DataFrame(tables[0])
df.columns=['Postcode','Borough','Neighbourhood']
df
df1=df.drop(0)


In [9]:
df1.head()


,Postcode,Borough,Neighbourhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


## Methodology 
In this project we will direct our efforts on analysing neighbourhood of toronto and see the top 10 most visited places and also their category


In first step we have collected the required data: about neighbourhoods fom wikipedia and then location and type (category) of every neighbourhood of toronto 10 most visted places.

we will use a map folium to cluster together things.

we might use heatmaps to denote areas that are dominated by a certain type of category


### removing not assigned from Borough and grouping by Postcode

## Analysis
Let's perform some basic explanatory data analysis and derive some additional info from our raw data.

In [34]:
df1=df1[df1.Borough != 'Not assigned']

df1.head(10)
df_new=pd.DataFrame(df1)
df_new.head()
df2=df1.groupby("Postcode").agg(lambda x:','.join(set(x)))
df2=df2.reset_index()
df2.head(20)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
2,M1E,Scarborough,"Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park"
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile"
8,M1M,Scarborough,"Scarborough Village West,Cliffcrest,Cliffside"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [11]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

len(df2[df2['Neighbourhood'] == 'Not assigned'])


0

In [12]:
df2.shape


(103, 3)

### getting the lat and lon data and combining it into data frame

In [13]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
y = df_data_1[df_data_1.columns[-2:]]


In [15]:
y.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [16]:
# place the DataFrames side by side
df = pd.concat([df2, y], axis=1)

In [17]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### folium, maps, representation

In [18]:
LIMIT=100
radius=500
CLIENT_ID = 'PVAM3HTW1ZF42X3LFABRZJCTHKHB5CISCRBKFJOPKHLPFXWJ' # your Foursquare ID
CLIENT_SECRET = 'KMOZIKQFGLIE4UOGXPL5SDDPFZ5ZMTNIJATVNBZKWA0BEZGB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: PVAM3HTW1ZF42X3LFABRZJCTHKHB5CISCRBKFJOPKHLPFXWJ
CLIENT_SECRET:KMOZIKQFGLIE4UOGXPL5SDDPFZ5ZMTNIJATVNBZKWA0BEZGB


In [19]:
def getNearbyVenues(names_list, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for names, lat, lng in zip(names_list, latitudes, longitudes):
        
        for name in names.split(","):
            
            print(name)
            
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
            results = requests.get(url).json()["response"]['groups'][0]['items']
           # print(requests.get(url).json())
            venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
            
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names_list=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Malvern
Rouge
Rouge Hill
Port Union
Highland Creek
Morningside
West Hill
Guildwood
Woburn
Cedarbrae
Scarborough Village
Ionview
Kennedy Park
East Birchmount Park
Oakridge
Clairlea
Golden Mile
Scarborough Village West
Cliffcrest
Cliffside
Birch Cliff
Cliffside West
Wexford Heights
Scarborough Town Centre
Dorset Park
Maryvale
Wexford
Agincourt
Sullivan
Tam O'Shanter
Clarks Corners
Milliken
L'Amoreaux East
Steeles East
Agincourt North
L'Amoreaux West
Upper Rouge
Hillcrest Village
Henry Farm
Fairview
Oriole
Bayview Village
Silver Hills
York Mills
Newtonbrook
Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Flemingdon Park
Bathurst Manor
Wilson Heights
Downsview North
York University
Northwood Park
Downsview East
CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill
Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
Riverdale
The Danforth West
India Bazaar
The Beac

In [21]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Malvern,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Rouge,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
3,Rouge,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
4,Rouge Hill,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping


In [22]:
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,30,30,30,30,30,30
Agincourt,4,4,4,4,4,4
Agincourt North,3,3,3,3,3,3
Albion Gardens,11,11,11,11,11,11
Alderwood,10,10,10,10,10,10
Bathurst Manor,18,18,18,18,18,18
Bathurst Quay,16,16,16,16,16,16
Bayview Village,4,4,4,4,4,4
Beaumond Heights,11,11,11,11,11,11


In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
mylt=list(toronto_onehot.columns)
#print(mylt)
mylt.remove('Neighborhood')
#print(mylt)

new_col=['Neighborhood']
new_col.extend(mylt)
#print(new_col)
toronto_onehot=toronto_onehot[new_col]

In [24]:
toronto_onehot.head(10)

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Malvern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Malvern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rouge Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Rouge Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Port Union,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Port Union,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Highland Creek,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Highland Creek,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.033333,0.0,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Agincourt,0.0,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Agincourt North,0.0,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Albion Gardens,0.0,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Alderwood,0.0,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Bathurst Manor,0.0,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bathurst Quay,0.0,0.062500,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Bayview Village,0.0,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Beaumond Heights,0.0,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Bedford Park,0.0,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
from sklearn.cluster import KMeans


In [27]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 2, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

In [28]:
toronto_grouped_clustering.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,...,0.0,0.033333,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [30]:
import numpy as np
num_top_venues = 10


indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Steakhouse,Café,Pizza Place,Asian Restaurant,Hotel,Concert Hall,Monument / Landmark,Plaza,Food Court,Lounge
1,Agincourt,Lounge,Breakfast Spot,Chinese Restaurant,Sandwich Place,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
2,Agincourt North,Park,Playground,Asian Restaurant,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
3,Albion Gardens,Grocery Store,Liquor Store,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Pizza Place,Beer Store,Video Store,Coffee Shop
4,Alderwood,Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Dance Studio,Sandwich Place,Coffee Shop,Fruit & Vegetable Store


In [31]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,4,Fast Food Restaurant,Print Shop,Yoga Studio,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
1,Malvern,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop,4,Fast Food Restaurant,Print Shop,Yoga Studio,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
2,Rouge,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,4,Fast Food Restaurant,Print Shop,Yoga Studio,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
3,Rouge,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop,4,Fast Food Restaurant,Print Shop,Yoga Studio,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
4,Rouge Hill,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping,4,Construction & Landscaping,Bar,Yoga Studio,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run


In [32]:
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip install folium
import folium

     |████████████████████████████████| 92kB 15.9MB/s eta 0:00:01


In [33]:
map_clusters = folium.Map(location=[43.65, -79.38], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Venue Latitude'].astype(int), toronto_merged['Venue Longitude'].astype(int), toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results and Discussion 
it is clear from the above tables and above maps what are the most visited categories and places visited in each neighbourhood and thereby its upto investor or business what neighbourhoods to target maybe for investments or for advertisments or maybe for voting campaigns.Applications of using this is endless. 

## Conclusion
it is upto the investor to invest. He now has something less to gamble while investing. he can open something of that category for which neighbourhood is famous for.